In [375]:
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
from scipy.stats import norm
import statsmodels.api as sm
import statsmodels.formula.api as smf
from mizani import transforms
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import statsmodels.nonparametric.kernel_regression as loess
warnings.filterwarnings("ignore")


In [376]:
# location folders
dirname = "E:/OneDrive - my.cuesta.edu/ceu/ml/Data_Analysis_3/PS1"
data_in = dirname + "/osfstorage-archive/clean/"

In [377]:
data_all = pd.read_csv(data_in + "morg-2014-emp.csv")

In [378]:
print(data_all.columns)
print(data_all.shape)

Index(['Unnamed: 0', 'hhid', 'intmonth', 'stfips', 'weight', 'earnwke',
       'uhours', 'grade92', 'race', 'ethnic', 'age', 'sex', 'marital',
       'ownchild', 'chldpres', 'prcitshp', 'state', 'ind02', 'occ2012',
       'class', 'unionmme', 'unioncov', 'lfsr94'],
      dtype='object')
(149316, 23)


In [379]:
print(np.sort(data_all["occ2012"].unique()))

[  10   20   40   50   60  100  110  120  135  136  137  140  150  160
  205  220  230  300  310  330  340  350  360  410  420  425  430  500
  510  520  530  540  565  600  630  640  650  700  710  725  726  735
  740  800  810  820  830  840  850  860  900  910  930  940  950 1005
 1006 1007 1010 1020 1030 1050 1060 1105 1106 1107 1200 1220 1240 1300
 1310 1320 1340 1350 1360 1400 1410 1420 1430 1440 1450 1460 1500 1510
 1520 1530 1540 1550 1560 1600 1610 1640 1650 1700 1710 1720 1740 1760
 1800 1820 1840 1860 1900 1910 1920 1930 1965 2000 2010 2015 2016 2025
 2040 2050 2060 2100 2105 2145 2160 2200 2300 2310 2320 2330 2340 2400
 2430 2440 2540 2550 2600 2630 2700 2710 2720 2740 2750 2760 2800 2810
 2825 2830 2840 2850 2860 2900 2910 2920 3000 3010 3030 3040 3050 3060
 3110 3140 3150 3160 3200 3210 3220 3230 3245 3250 3255 3256 3258 3260
 3300 3310 3320 3400 3420 3500 3510 3520 3535 3540 3600 3610 3620 3630
 3640 3645 3646 3647 3648 3649 3655 3700 3710 3720 3730 3740 3750 3800
 3820 

In [380]:
# 4820 Securities, commodities, and financial services sales agents
# 5165 Financial clerks, all other
# 0120 Financial managers
# 0800  - 0950

# print(data_all.dtypes)

# Use the combined condition to select the rows
data = data_all.query("occ2012 == 4820 or occ2012 == 5165 or occ2012 == 120 or (occ2012>799 and occ2012<951)")
np.sort(data["occ2012"].unique())


array([ 120,  800,  810,  820,  830,  840,  850,  860,  900,  910,  930,
        940,  950, 4820, 5165], dtype=int64)

In [381]:
data= data[
    [
        "lfsr94",
        "earnwke",
        "uhours",
        "grade92",
        "race",
        "ethnic",
        "age",
        "sex",
        "marital",
        "ownchild",
        "chldpres",
        "prcitshp",
        "state",
        "ind02",
        "occ2012",
        "class",
        "unionmme", 
        "unioncov"]
]

In [382]:
np.sort(data["ind02"].unique())


array(['** Alcoholic beverages, merchant wholesalers (*4248)',
       '** Construction (23)',
       '** Drugs, sundries, and chemical and allied products, merchant (wholesalerss*4242, 4246)',
       '** Farm product raw materials, merchant wholesalers (*4245)',
       '** Farm supplies, merchant wholesalers (*42491)',
       '** Groceries and related products, merchant wholesalers (*4244)',
       '** Lumber and other construction materials, merchant wholesalers (*4233)',
       '** Machinery, equipment, and supplies, merchant wholesalers (*4238)',
       '** Metals and minerals, except petroleum, merchant wholesalers (*4235)',
       '** Miscellaneous (nondurable goods, merchant wholesalerss*4249 exc. 42491)',
       '** Miscellaneous durable goods, merchant wholesalers (*4239 exc. 42393)',
       '** Motor vehicles, parts and supplies, merchant wholesalers (*4231)',
       '** Petroleum and petroleum products, merchant wholesalers (*4247)',
       '** Professional and commercial equ

In [383]:
for i in data.index:
    data.at[i,"ind02"] = data.at[i,"ind02"].replace("*", "")
    data.at[i,"ind02"] = data.at[i,"ind02"].replace(" ", "")
np.sort(data["ind02"].unique())


array(['3095', '3365', '3895', '4195', '4265', '4795', '5275',
       'Accounting,taxpreparation,bookkeeping,andpayrollservices(5412)',
       'Administrationofeconomicprogramsandspaceresearch(926,927)',
       'Administrationofenvironmentalqualityandhousingprograms(924,925)',
       'Administrationofhumanresourceprograms(923)',
       'Advertisingandrelatedservices(5418)',
       'Aerospaceproductsandpartsmanufacturing(336414,336415,336419)',
       'Agriculturalchemicalmanufacturing(3253)',
       'Agriculturalimplementmanufacturing(33311)',
       'Aircraftandpartsmanufacturing(336411to336413)',
       'Airtransportation(481)',
       'Alcoholicbeverages,merchantwholesalers(4248)',
       'Aluminumproductionandprocessing(3313)',
       'Animalfood,grainandoilseedmilling(3111,3112)',
       'Animalproduction(112)', 'Animalslaughteringandprocessing(3116)',
       'Architectural,engineering,andrelatedservices(5413)',
       'Automobiledealers(4411)',
       'Automotiveequipmentrentalan

In [384]:
for i in data.index:
    data.at[i, "ind01"] = data.at[i, "ind02"][0:3]

In [385]:
dummies = pd.get_dummies(data["ind01"], prefix="ind01", drop_first=True)

In [386]:
data = pd.concat([data, dummies], axis=1)

In [387]:
data.columns

Index(['lfsr94', 'earnwke', 'uhours', 'grade92', 'race', 'ethnic', 'age',
       'sex', 'marital', 'ownchild',
       ...
       'ind01_Tob', 'ind01_Toy', 'ind01_Tra', 'ind01_Tru', 'ind01_Vet',
       'ind01_War', 'ind01_Was', 'ind01_Wat', 'ind01_Who', 'ind01_Wir'],
      dtype='object', length=161)

In [388]:
data["wage"] = data["earnwke"] / data["uhours"]

In [389]:
data["ln_wage"] = np.log(data["wage"])

In [390]:
data["age2"] = np.power(data["age"], 2)

In [391]:
print(np.sort(data["grade92"].unique()))

[31 32 34 35 36 37 38 39 40 41 42 43 44 45 46]


In [392]:
dict = {31:0, 32: (1+2+3+4)/4, 34:(7+8)/2, 35:9, 36:10, 37:11, 38:12, 39:13, 40:14, 41:15, 42:15, 43:16, 44:18, 45:23, 46:24,}
data=data.replace({"grade92": dict})
print(np.sort(data["grade92"].unique()))

[ 0.   2.5  7.5  9.  10.  11.  12.  13.  14.  15.  16.  18.  23.  24. ]


In [393]:
print(np.sort(data["race"].unique()))

[ 1  2  3  4  5  6  7  8  9 10 11 13 16 19 21]


In [394]:
dummies = pd.get_dummies(data["race"], prefix="race", drop_first=True)
data = pd.concat([data, dummies], axis=1)

In [395]:
dict = {1:1, 2: 0}
data=data.replace({"sex": dict})
print(np.sort(data["sex"].unique()))

[0 1]


In [396]:
print(np.sort(data["marital"].unique()))

[1 2 3 4 5 6 7]


In [397]:
dummies = pd.get_dummies(data["marital"], prefix="marital", drop_first=True)
data = pd.concat([data, dummies], axis=1)

In [398]:
print(data["unioncov"].unique())

['No' nan 'Yes']


In [399]:
dict = {"No":0, "Yes": 1}
data=data.replace({"unioncov": dict})
print(data["unioncov"].unique())

[ 0. nan  1.]


In [400]:
print(data["unionmme"].unique())

['No' 'Yes']


In [401]:
dict = {"No":0, "Yes": 1}
data=data.replace({"unionmme": dict})
print(data["unionmme"].unique())

[0 1]


In [402]:
print(data["ownchild"].unique())

[0 1 2 3 4 5 6]


In [403]:
print(data["lfsr94"].unique())

['Employed-At Work' 'Employed-Absent']


In [404]:
dict = {"Employed-At Work":1, "Employed-Absent": 0}
data=data.replace({"lfsr94": dict})
print(data["lfsr94"].unique())

[1 0]


In [405]:
print(data["class"].unique())

['Private, For Profit' 'Private, Nonprofit' 'Government - State'
 'Government - Local' 'Government - Federal']


In [406]:
dummies = pd.get_dummies(data["class"], prefix="class", drop_first=True)
data = pd.concat([data, dummies], axis=1)

In [407]:
print(data["occ2012"].unique())
dummies = pd.get_dummies(data["occ2012"], prefix="occ2012", drop_first=True)
data = pd.concat([data, dummies], axis=1)

[ 120  800  910  850  820 4820  840  860 5165  810  940  950  900  930
  830]


In [408]:
print(data["state"].unique())
dummies = pd.get_dummies(data["state"], prefix="state", drop_first=True)
data = pd.concat([data, dummies], axis=1)

['63' '94' '86' '71' '93' '84' 'CT' '51' '53' '59' '58' '95' '82' 'IL'
 'IN' 'IA' '47' '61' '72' 'ME' '52' 'MA' 'MI' 'MN' '64' 'MO' '81' '46'
 '88' 'NH' 'NJ' '85' 'NY' '56' 'ND' 'OH' '73' '92' 'PA' 'RI' '57' 'SD'
 '62' '74' '87' 'VT' '54' '91' 'WI' '83' '55' '6' '9' '8' '7' '1' '5' '3'
 '4' '2' 4 6 7 1 5 3 8 2 9]


In [409]:
print(data["prcitshp"].unique())
dummies = pd.get_dummies(data["prcitshp"], prefix="prcitshp", drop_first=True)
data = pd.concat([data, dummies], axis=1)

['Native, Born In US' 'Native, Born Abroad Of US Parent(s)'
 'Foreign Born, US Cit By Naturalization'
 'Native, Born in PR or US Outlying Area' 'Foreign Born, Not a US Citizen']


In [410]:
data["exp"] = data["age"] - data["grade92"] - 6
data["exp2"] = data["exp"]*data["exp"]

In [411]:
print(data["ethnic"].unique())
dummies = pd.get_dummies(data["ethnic"], prefix="ethnic", drop_first=True)
data = pd.concat([data, dummies], axis=1)

[nan  1.  8.  7.  2.  3.  6.  5.  4.]


In [412]:
data.columns.tolist()

['lfsr94',
 'earnwke',
 'uhours',
 'grade92',
 'race',
 'ethnic',
 'age',
 'sex',
 'marital',
 'ownchild',
 'chldpres',
 'prcitshp',
 'state',
 'ind02',
 'occ2012',
 'class',
 'unionmme',
 'unioncov',
 'ind01',
 'ind01_336',
 'ind01_389',
 'ind01_419',
 'ind01_426',
 'ind01_479',
 'ind01_527',
 'ind01_Acc',
 'ind01_Adm',
 'ind01_Adv',
 'ind01_Aer',
 'ind01_Agr',
 'ind01_Air',
 'ind01_Alc',
 'ind01_Alu',
 'ind01_Ani',
 'ind01_Arc',
 'ind01_Aut',
 'ind01_Bak',
 'ind01_Ban',
 'ind01_Bee',
 'ind01_Bev',
 'ind01_Boo',
 'ind01_Bui',
 'ind01_Bus',
 'ind01_Car',
 'ind01_Cem',
 'ind01_Chi',
 'ind01_Civ',
 'ind01_Clo',
 'ind01_Coa',
 'ind01_Col',
 'ind01_Com',
 'ind01_Con',
 'ind01_Cou',
 'ind01_Cro',
 'ind01_Cut',
 'ind01_Dai',
 'ind01_Dat',
 'ind01_Dep',
 'ind01_Dri',
 'ind01_Dru',
 'ind01_Ele',
 'ind01_Emp',
 'ind01_Eng',
 'ind01_Exe',
 'ind01_Far',
 'ind01_Fis',
 'ind01_Foo',
 'ind01_For',
 'ind01_Fou',
 'ind01_Fru',
 'ind01_Fue',
 'ind01_Fun',
 'ind01_Fur',
 'ind01_Gas',
 'ind01_Gif',
 'ind

In [413]:
data_r = data.drop(["earnwke","ind01","ind02", "state", "class", "occ2012", "prcitshp", "marital", "ethnic", "race"],axis=1)

In [464]:
data_r.columns.tolist()

['lfsr94',
 'uhours',
 'grade92',
 'age',
 'sex',
 'ownchild',
 'chldpres',
 'unionmme',
 'unioncov',
 'ind01_336',
 'ind01_389',
 'ind01_419',
 'ind01_426',
 'ind01_479',
 'ind01_527',
 'ind01_Acc',
 'ind01_Adm',
 'ind01_Adv',
 'ind01_Aer',
 'ind01_Agr',
 'ind01_Air',
 'ind01_Alc',
 'ind01_Alu',
 'ind01_Ani',
 'ind01_Arc',
 'ind01_Aut',
 'ind01_Bak',
 'ind01_Ban',
 'ind01_Bee',
 'ind01_Bev',
 'ind01_Boo',
 'ind01_Bui',
 'ind01_Bus',
 'ind01_Car',
 'ind01_Cem',
 'ind01_Chi',
 'ind01_Civ',
 'ind01_Clo',
 'ind01_Coa',
 'ind01_Col',
 'ind01_Com',
 'ind01_Con',
 'ind01_Cou',
 'ind01_Cro',
 'ind01_Cut',
 'ind01_Dai',
 'ind01_Dat',
 'ind01_Dep',
 'ind01_Dri',
 'ind01_Dru',
 'ind01_Ele',
 'ind01_Emp',
 'ind01_Eng',
 'ind01_Exe',
 'ind01_Far',
 'ind01_Fis',
 'ind01_Foo',
 'ind01_For',
 'ind01_Fou',
 'ind01_Fru',
 'ind01_Fue',
 'ind01_Fun',
 'ind01_Fur',
 'ind01_Gas',
 'ind01_Gif',
 'ind01_Gla',
 'ind01_Gro',
 'ind01_Har',
 'ind01_Hea',
 'ind01_Hom',
 'ind01_Hos',
 'ind01_Hou',
 'ind01_Ind',
 '

In [513]:
columns = data_r.columns.to_list()
state_all = filter(lambda x: x.startswith('state_'), columns)
ind_all = filter(lambda x: x.startswith('ind01_'), columns)
ethnic_all = filter(lambda x: x.startswith('ethnic_'), columns)

In [515]:
formula = 'wage ~ ' + ' + '.join(ind_all)
model = smf.ols(formula, data=data_r)
results = model.fit()

# Print the results
print(results.summary())


PatsyError: expected a noun, but instead the expression ended
    wage ~
         ^